In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark=SparkSession.builder.config("spark.driver.host",'localhost').getOrCreate()

In [2]:
spark

In [53]:

from pyspark.sql.types import StructType, StructField, StringType, IntegerType



# Sample data with 20 rows and 4 columns
data = [
    ("Alice", 25, "Engineer", "A"),
    ("Bob", 30, "Analyst", "B"),
    ("Charlie", 22, "Intern", "C"),
    ("David", 35, "Manager", "A"),
    ("Eve", 28, "Developer", "B"),
    ("Frank", 32, "Designer", "C"),
    ("Grace", 26, "Scientist", "A"),
    ("Harry", 40, "Manager", "B"),
    ("Ivy", 31, "Analyst", "C"),
    ("Jack", 29, "Engineer", "A"),
    ("Kate", 27, "Developer", "B"),
    ("Leo", 33, "Designer", "C"),
    ("Mia", 24, "Scientist", "A"),
    ("Nathan", 38, "Engineer", "B"),
    ("Olivia", 23, "Analyst", "C"),
    ("Peter", 36, "Manager", "A"),
    ("Quinn", 34, "Scientist", "B"),
    ("Rachel", 39, "Developer", "C"),
    ("Sam", 37, "Designer", "A"),
    ("Tina", 21, "Intern", "B")
]

# Define schema
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Occupation", StringType(), True),
    StructField("Group", StringType(), True)
])

# Create a DataFrame
df = spark.createDataFrame(data, schema=schema)

# Show the original DataFrame
print("Original DataFrame:")
df.show()

Original DataFrame:
+-------+---+----------+-----+
|   Name|Age|Occupation|Group|
+-------+---+----------+-----+
|  Alice| 25|  Engineer|    A|
|    Bob| 30|   Analyst|    B|
|Charlie| 22|    Intern|    C|
|  David| 35|   Manager|    A|
|    Eve| 28| Developer|    B|
|  Frank| 32|  Designer|    C|
|  Grace| 26| Scientist|    A|
|  Harry| 40|   Manager|    B|
|    Ivy| 31|   Analyst|    C|
|   Jack| 29|  Engineer|    A|
|   Kate| 27| Developer|    B|
|    Leo| 33|  Designer|    C|
|    Mia| 24| Scientist|    A|
| Nathan| 38|  Engineer|    B|
| Olivia| 23|   Analyst|    C|
|  Peter| 36|   Manager|    A|
|  Quinn| 34| Scientist|    B|
| Rachel| 39| Developer|    C|
|    Sam| 37|  Designer|    A|
|   Tina| 21|    Intern|    B|
+-------+---+----------+-----+



In [54]:
import pandas as pd

# Creating two sample DataFrames
df1 = pd.DataFrame({
    'ID': [1, 2, 3, 4],
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Age': [25, 30, 35, 40]
})

df2 = pd.DataFrame({
    'ID': [1, 6, 3, 4],
    'Salary': [50000, 60000, 70000, 80000],
    'Department': ['HR', 'IT', 'Finance', 'Marketing']
})

# Performing a join on multiple columns using where() and filter()
merged_df = df1.where(df1['ID'] == df2['ID']).filter(['ID', 'Name', 'Age']). \
    join(df2.where(df1['ID'] == df2['ID']).filter(['Salary', 'Department']))
print(df1.where(df1['ID'] == df2['ID']).filter(['ID', 'Name', 'Age']))
# s

# Displaying the result
print(merged_df)

    ID     Name   Age
0  1.0    Alice  25.0
1  NaN      NaN   NaN
2  3.0  Charlie  35.0
3  4.0    David  40.0
    ID     Name   Age   Salary Department
0  1.0    Alice  25.0  50000.0         HR
1  NaN      NaN   NaN      NaN        NaN
2  3.0  Charlie  35.0  70000.0    Finance
3  4.0    David  40.0  80000.0  Marketing


In [55]:
df1 = spark.createDataFrame([(1, "Alice"), (2, "Bob"), (3, "Charlie")], ["id", "name"])
df2 = spark.createDataFrame([(1, "Math"), (3, "English")], ["id", "subject"])

# Perform a left outer join on the "id" column
result = df1.join(df2, df1["id"] == df2["id"], "left_outer")
result2 = df1.join(df2, df1["id"] == df2["id"], "left")
result3 = df1.join(df2, df1["id"] == df2["id"], "leftsemi")
result4 = df1.join(df2, df1["id"] == df2["id"], "leftanti")

# Show the result
result.show()
result2.show()
result3.show()
result4.show()

+---+-------+----+-------+
| id|   name|  id|subject|
+---+-------+----+-------+
|  1|  Alice|   1|   Math|
|  2|    Bob|null|   null|
|  3|Charlie|   3|English|
+---+-------+----+-------+

+---+-------+----+-------+
| id|   name|  id|subject|
+---+-------+----+-------+
|  1|  Alice|   1|   Math|
|  2|    Bob|null|   null|
|  3|Charlie|   3|English|
+---+-------+----+-------+

+---+-------+
| id|   name|
+---+-------+
|  1|  Alice|
|  3|Charlie|
+---+-------+

+---+----+
| id|name|
+---+----+
|  2| Bob|
+---+----+



In [56]:

from pyspark.sql.functions import transform
from pyspark.sql.functions import col, upper
from pyspark.sql.types import StringType,StructField,StructType,ArrayType

# Sample DataFrame
data1 = [(1, ["apple", "orange"]), (2, ["banana", "grape"]), (3, ["cherry", "pear"])]

# Define the schema
schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", ArrayType(StringType()), True)
])

# Create DataFrame with the corrected data
df = spark.createDataFrame(data1, schema=schema)
df.show()

# Apply the transformation using withColumn
transformed_df = df.withColumn("name", transform("name", lambda x: upper(x)).alias("name_upper"))

# Show the result
transformed_df.show()

+---+---------------+
| id|           name|
+---+---------------+
|  1|[apple, orange]|
|  2|[banana, grape]|
|  3| [cherry, pear]|
+---+---------------+

+---+---------------+
| id|           name|
+---+---------------+
|  1|[APPLE, ORANGE]|
|  2|[BANANA, GRAPE]|
|  3| [CHERRY, PEAR]|
+---+---------------+



In [57]:

from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Sample data
data = [("John", "Male"), ("Alice", "Female"), ("Bob", "Male")]
schema = ["name", "gender"]

# Create a DataFrame
df = spark.createDataFrame(data, schema=schema)

# List of transformations
indexer = StringIndexer(inputCol="gender", outputCol="gender_index")
encoder = OneHotEncoder(inputCol="gender_index", outputCol="gender_onehot")
assembler = VectorAssembler(inputCols=["gender_onehot"], outputCol="features")

# Fit and transform each transformation
indexer_model = indexer.fit(df)
df_indexed = indexer_model.transform(df)

encoder_model = encoder.fit(df_indexed)
df_encoded = encoder_model.transform(df_indexed)

df_assembled = assembler.transform(df_encoded)

# Show the result
df_assembled.show()


+-----+------+------------+-------------+--------+
| name|gender|gender_index|gender_onehot|features|
+-----+------+------------+-------------+--------+
| John|  Male|         0.0|(1,[0],[1.0])|   [1.0]|
|Alice|Female|         1.0|    (1,[],[])|   [0.0]|
|  Bob|  Male|         0.0|(1,[0],[1.0])|   [1.0]|
+-----+------+------------+-------------+--------+



In [65]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,upper
from pyspark.sql.types import StringType



# Sample data
data = [("John", "Male"), ("Alice", "Female"), ("Bob", "Male")]
schema = ["name", "gender"]

# Create a DataFrame
df = spark.createDataFrame(data, schema=schema)

# User-defined function
@udf(StringType())
def concatenate_name_gender(name, gender):
    return f"{name.upper()} ({gender})"

# Apply the UDF
df_result = df.withColumn("full_name_gender", concatenate_name_gender(df["name"], df["gender"]))

# Show the result
df_result.show()


+-----+------+----------------+
| name|gender|full_name_gender|
+-----+------+----------------+
| John|  Male|     JOHN (Male)|
|Alice|Female|  ALICE (Female)|
|  Bob|  Male|      BOB (Male)|
+-----+------+----------------+



In [62]:
spark

In [68]:
from pyspark.sql.functions import udf,upper

def up(x):
    return x.upper()
upperCaseUDF = udf(up, StringType())
# change is the name with which we can access this fucntion, this is the registered name for upperCase function.

data = [("John", "Male"), ("Alice", "Female"), ("Bob", "Male")]
schema = ["Name", "gender"]

# Create a DataFrame
df = spark.createDataFrame(data, schema=schema)
# Custom UDF with withColumn()
df.show()

df.withColumn("Created Name", upperCaseUDF(col("Name"))).show(truncate=False)




+-----+------+
| Name|gender|
+-----+------+
| John|  Male|
|Alice|Female|
|  Bob|  Male|
+-----+------+

+-----+------+------------+
|Name |gender|Created Name|
+-----+------+------------+
|John |Male  |JOHN        |
|Alice|Female|ALICE       |
|Bob  |Male  |BOB         |
+-----+------+------------+



In [72]:
from pyspark.sql.functions import udf,upper,lower

def up(x):
    return [x.upper(),x.lower()]
upperCaseUDF = udf(up,ArrayType(StringType()))
# change is the name with which we can access this fucntion, this is the registered name for upperCase function.

data = [("John", "Male"), ("Alice", "Female"), ("Bob", "Male")]
schema = ["Name", "gender"]

# Create a DataFrame
df = spark.createDataFrame(data, schema=schema)
# Custom UDF with withColumn()
df.show()

df.withColumn("Created Name", upperCaseUDF(col("Name"))).show(truncate=False)

+-----+------+
| Name|gender|
+-----+------+
| John|  Male|
|Alice|Female|
|  Bob|  Male|
+-----+------+

+-----+------+--------------+
|Name |gender|Created Name  |
+-----+------+--------------+
|John |Male  |[JOHN, john]  |
|Alice|Female|[ALICE, alice]|
|Bob  |Male  |[BOB, bob]    |
+-----+------+--------------+



In [73]:
from pyspark.sql.functions import udf,upper


upperCaseUDF = udf(lambda x:upper(), StringType())
# change is the name with which we can access this fucntion, this is the registered name for upperCase function.

data = [("John", "Male"), ("Alice", "Female"), ("Bob", "Male")]
schema = ["Name", "gender"]

# Create a DataFrame
df = spark.createDataFrame(data, schema=schema)
# Custom UDF with withColumn()
df.show()

df.withColumn("Created Name", upperCaseUDF(col("Name"))).show(truncate=False)

+-----+------+
| Name|gender|
+-----+------+
| John|  Male|
|Alice|Female|
|  Bob|  Male|
+-----+------+



PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Users\itsrk\AppData\Local\Temp\ipykernel_15160\2047332330.py", line 4, in <lambda>
  File "C:\Program Files\spark-3.4.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\sql\utils.py", line 160, in wrapped
    return f(*args, **kwargs)
TypeError: upper() missing 1 required positional argument: 'col'


In [78]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split



# Sample DataFrame
data = [("John,Doe", 25), ("Alice,Smith", 30), ("Bob,Johnson", 22)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)
df.show()

# Split the "Name" column by commas and apply flatMap
df_flatmap = df.withColumn("Names", split(col("Name"), ",")) \
    .select("Age", "Names") \
    .rdd \
    .flatMap(lambda row: [(row.Age, name.strip()) for name in row.Names])
df_flatmap.toDF.show()
# Display the result
for row in df_flatmap.collect():
    print(row)

+-----------+---+
|       Name|Age|
+-----------+---+
|   John,Doe| 25|
|Alice,Smith| 30|
|Bob,Johnson| 22|
+-----------+---+



AttributeError: 'function' object has no attribute 'show'

In [92]:
from pyspark.sql import SparkSession
from pyspark.sql import Row


# Example data
data = ["Hello World", "PySpark Map FlatMap Example", "OpenAI GPT-3"]

# Create an RDD from the data
rdd = spark.sparkContext.parallelize(data)

# Example using map transformation
# Transform each element by splitting the words
map_result = rdd.map(lambda sentence: sentence.split(" "))

# Convert map_result into a DataFrame
map_df = map_result.map(lambda x: Row(words=x)).toDF()

# Show the map DataFrame
print("Map DataFrame:")
map_df.show(truncate=False)

# Example using flatMap transformation
# Transform each element by splitting the words and flatten the results
flatMap_result = rdd.flatMap(lambda sentence: sentence.split(" "))

# Convert flatMap_result into a DataFrame
flatMap_df = flatMap_result.map(lambda x: Row(word=x)).toDF()

# Show the flatMap DataFrame
print("\nFlatMap DataFrame:")
flatMap_df.show(truncate=False)


Map DataFrame:
+--------------------------------+
|words                           |
+--------------------------------+
|[Hello, World]                  |
|[PySpark, Map, FlatMap, Example]|
|[OpenAI, GPT-3]                 |
+--------------------------------+


FlatMap DataFrame:
+-------+
|word   |
+-------+
|Hello  |
|World  |
|PySpark|
|Map    |
|FlatMap|
|Example|
|OpenAI |
|GPT-3  |
+-------+



In [96]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Sample DataFrame
data = [("John", 25), ("Alice", 30), ("Bob", 22)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)

# Define a function to print each row
def print_row(row):
    print("Processing row:", row)

# Apply foreach to print each row
df.rdd.foreach(print_row)



In [106]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


# Sample data
data = [("John", "Male", 10),
        ("Alice", "Female", 20),
        ("Bob", "Male", 10),
        ("John", "Female", 30),
        ("Bob", "Female", 20),
        ("Bob", "Male", 40)]

columns = ["name", "gender", "age_group"]

# Create a DataFrame
df = spark.createDataFrame(data, columns)

# Pivot the DataFrame
pivot_df = df.groupBy("name").pivot("gender").sum("age_group")

# Show the result
pivot_df.show()


+-----+------+----+
| name|Female|Male|
+-----+------+----+
|  Bob|    20|  50|
| John|    30|  10|
|Alice|    20|null|
+-----+------+----+



In [3]:
match_data = [
    ("Jadeja", 'Wankhede', 40, 2, 1),
    ("Hardik", 'Eden Gardens', 55, 0, 2),
    ("Jadeja", 'Eden Gardens', 25, 3, 0),
    ("Watson", 'Wankhede', 30, 1, 0),
    ("Hardik", 'Wankhede', 45, 0, 1),
    ("Jadeja", 'Wankhede', 60, 1, 1),
    ("Hardik", 'Eden Gardens', 45, 0, 3),
    ("Hardik", 'Eden Gardens', 30, 4, 0),
    ("Watson", 'Wankhede', 20, 2, 2),
    ("Watson", 'Eden Gardens', 50, 0, 0)
]


match_schema = "Player_Name String , Stadium String , Runs int , Wickets int , Catches int"
match_df = spark.createDataFrame(data = match_data, schema = match_schema)

match_df.show()

+-----------+------------+----+-------+-------+
|Player_Name|     Stadium|Runs|Wickets|Catches|
+-----------+------------+----+-------+-------+
|     Jadeja|    Wankhede|  40|      2|      1|
|     Hardik|Eden Gardens|  55|      0|      2|
|     Jadeja|Eden Gardens|  25|      3|      0|
|     Watson|    Wankhede|  30|      1|      0|
|     Hardik|    Wankhede|  45|      0|      1|
|     Jadeja|    Wankhede|  60|      1|      1|
|     Hardik|Eden Gardens|  45|      0|      3|
|     Hardik|Eden Gardens|  30|      4|      0|
|     Watson|    Wankhede|  20|      2|      2|
|     Watson|Eden Gardens|  50|      0|      0|
+-----------+------------+----+-------+-------+



In [7]:
pivot_df=match_df.groupBy('Player_Name').pivot('Stadium').sum("Runs","Wickets")
pivot_df.show()

+-----------+----------------------+-------------------------+------------------+---------------------+
|Player_Name|Eden Gardens_sum(Runs)|Eden Gardens_sum(Wickets)|Wankhede_sum(Runs)|Wankhede_sum(Wickets)|
+-----------+----------------------+-------------------------+------------------+---------------------+
|     Jadeja|                    25|                        3|               100|                    3|
|     Watson|                    50|                        0|                50|                    3|
|     Hardik|                   130|                        4|                45|                    0|
+-----------+----------------------+-------------------------+------------------+---------------------+



In [13]:
pivot_df=match_df.groupBy('Player_Name').pivot('Stadium',['Eden Garden']).sum("Runs","Wickets")
pivot_df.show()

+-----------+---------------------+------------------------+
|Player_Name|Eden Garden_sum(Runs)|Eden Garden_sum(Wickets)|
+-----------+---------------------+------------------------+
|     Jadeja|                 null|                    null|
|     Watson|                 null|                    null|
|     Hardik|                 null|                    null|
+-----------+---------------------+------------------------+



In [12]:
pivot_df=match_df.groupBy('Player_Name').pivot('Runs').sum("Wickets")
pivot_df.show()

+-----------+----+----+----+----+----+----+----+----+
|Player_Name|  20|  25|  30|  40|  45|  50|  55|  60|
+-----------+----+----+----+----+----+----+----+----+
|     Jadeja|null|   3|null|   2|null|null|null|   1|
|     Watson|   2|null|   1|null|null|   0|null|null|
|     Hardik|null|null|   4|null|   0|null|   0|null|
+-----------+----+----+----+----+----+----+----+----+



In [15]:
cricket_data = [("Virat Kohli", 85, 100, 75),
        ("Steve Smith", 90, 105, 80),
        ("Kane Williamson", 88, 95, 70)]


df = spark.createDataFrame(cricket_data, ["Player", "Match1", "Match2", "Match3"])

df.show()


+---------------+------+------+------+
|         Player|Match1|Match2|Match3|
+---------------+------+------+------+
|    Virat Kohli|    85|   100|    75|
|    Steve Smith|    90|   105|    80|
|Kane Williamson|    88|    95|    70|
+---------------+------+------+------+



In [16]:
from pyspark.sql.functions import expr

In [27]:
stack_expr="stack(3,'Match11',Match1, 'Match22',Match2, 'Match33',Match3) as (Match,Score) "

In [28]:
unpivot_df=df.select("Player",expr(stack_expr))
unpivot_df.show()

+---------------+-------+-----+
|         Player|  Match|Score|
+---------------+-------+-----+
|    Virat Kohli|Match11|   85|
|    Virat Kohli|Match22|  100|
|    Virat Kohli|Match33|   75|
|    Steve Smith|Match11|   90|
|    Steve Smith|Match22|  105|
|    Steve Smith|Match33|   80|
|Kane Williamson|Match11|   88|
|Kane Williamson|Match22|   95|
|Kane Williamson|Match33|   70|
+---------------+-------+-----+



In [31]:

from pyspark.sql.functions import posexplode


# Example DataFrame
data = [(1, {"A": 10, "B": 20}),
        (2, {"A": 15, "C": 25, "D": 30}),
        (3, {"B": 22, "D": 35})]

columns = ["id", "dict_column"]

df = spark.createDataFrame(data, columns)

# Explode the dictionary column using posexplode
df_exploded = df.select("id", posexplode("dict_column").alias("position", "key", "value"))

# Show the result
df_exploded.show()


+---+--------+---+-----+
| id|position|key|value|
+---+--------+---+-----+
|  1|       0|  A|   10|
|  1|       1|  B|   20|
|  2|       0|  A|   15|
|  2|       1|  C|   25|
|  2|       2|  D|   30|
|  3|       0|  D|   35|
|  3|       1|  B|   22|
+---+--------+---+-----+



In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

data=[["1","2020-02-01"],["2","2019-03-01"],["3","2021-03-01"]]
df=spark.createDataFrame(data,["id","input"])
df.show()

+---+----------+
| id|     input|
+---+----------+
|  1|2020-02-01|
|  2|2019-03-01|
|  3|2021-03-01|
+---+----------+



In [34]:
df.select(col("input"),date_format(col("input"),"dd/MM/yyyy").alias("formatted_date")).show()

+----------+--------------+
|     input|formatted_date|
+----------+--------------+
|2020-02-01|    01/02/2020|
|2019-03-01|    01/03/2019|
|2021-03-01|    01/03/2021|
+----------+--------------+

